In [1]:
import torch ## Были проблемы с torch т.к мы используем ROCm вместо CUDA: проверяем, определена ли видеокарта
torch.cuda.is_available()

True

In [2]:
import os ## "оптимизированная" загрузка данных 
import pandas as pd
directory = '/home/maxem/Downloads/aclImdb/train/pos'  # Replace with the actual directory path

data = []
for filename in os.listdir(directory):
    if filename.endswith('.txt'):
        filepath = os.path.join(directory, filename)
        with open(filepath, 'r') as file:
            review = file.read()
            id, rating = filename.split('_')
            rating = int(rating.split(".")[0])
            data.append({'id': id, 'review': review, 'rating': rating, 'outcome': 1})

df_pos = pd.DataFrame(data)


directory = '/home/maxem/Downloads/aclImdb/train/neg'  # Replace with the actual directory path

data = []
for filename in os.listdir(directory):
    if filename.endswith('.txt'):
        filepath = os.path.join(directory, filename)
        with open(filepath, 'r') as file:
            review = file.read()
            id, rating = filename.split('_')
            rating = int(rating.split(".")[0])
            data.append({'id': id, 'review': review, 'rating': rating, 'outcome': 0})

df_neg = pd.DataFrame(data)

directory = '/home/maxem/Downloads/aclImdb/test/neg'  # Replace with the actual directory path

data = []
for filename in os.listdir(directory):
    if filename.endswith('.txt'):
        filepath = os.path.join(directory, filename)
        with open(filepath, 'r') as file:
            review = file.read()
            id, rating = filename.split('_')
            rating = int(rating.split(".")[0])
            data.append({'id': id, 'review': review, 'rating': rating, 'outcome': 0})

df_neg_test = pd.DataFrame(data)

directory = '/home/maxem/Downloads/aclImdb/test/pos'  # Replace with the actual directory path

data = []
for filename in os.listdir(directory):
    if filename.endswith('.txt'):
        filepath = os.path.join(directory, filename)
        with open(filepath, 'r') as file:
            review = file.read()
            id, rating = filename.split('_')
            rating = int(rating.split(".")[0])
            data.append({'id': id, 'review': review, 'rating': rating, 'outcome': 0})

df_pos_test = pd.DataFrame(data)

In [3]:
df = pd.concat([df_pos, df_neg, df_pos_test, df_neg_test], axis=0).sample(frac=1).reset_index(drop=True) ## shuffling, "смешивание" данных
df = df.astype({"id": str, "review": str, "rating": float})
df['rating'] = df['rating'].replace({1: 0, 2: 1, 3: 2, 4: 3, 7:4, 8:5, 9:6, 10:7})##Необходимое изменение, иначе CUDA выдаст ошибку (необходимо присутствие класса 0)
df

,id,review,rating,outcome
0,1015,"I wait for each new episode, each re-run with ...",7.0,1
1,8851,I have always been somewhat underwhelmed by Jo...,0.0,0
2,7825,It is to typical of people complaining about s...,7.0,0
3,2966,"No matter what you've heard, ""Fame"" is not a g...",1.0,0
4,7198,"This is one of the films that killed the ""spag...",0.0,0
...,...,...,...,...
49995,6460,The reason why this movie isn't any better kno...,5.0,0
49996,11465,"Without a doubt, this is one of the worst pict...",0.0,0
49997,566,"I loved the first 15 minutes, and I loved some...",2.0,0
49998,10907,Was flipping around the TV and HBO was showing...,6.0,1


In [ ]:
!pip3 install --force-reinstall torch torchvision torchaudio --index-url https://download.pytorch.org/whl/rocm5.4.2 -q ## установка torch для rocm

In [4]:
import numpy as np
import pandas as pd
from datasets import load_dataset, Dataset, ClassLabel, Value, Features
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer, DataCollatorWithPadding, AutoTokenizer ## использование очень удобной библиотеки Transformers 

2023-08-04 14:29:33.368185: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [5]:
model_id = "microsoft/deberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_id)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
model = AutoModelForSequenceClassification.from_pretrained(model_id, num_labels=8)

def preprocess(dataset):
    return tokenizer(dataset["text"], truncation=True)

Some weights of DebertaForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-base and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
dataset = Dataset.from_pandas(df)
dataset = dataset.remove_columns(["id", "outcome"])
dataset = dataset.rename_column("review", "text")
dataset = dataset.rename_column("rating", "label")
tokenized_text = dataset.map(preprocess, batched=True)
tokenized_text_split = tokenized_text.train_test_split(test_size=0.2) ##Вместо распределения тренировочного\тестировочного распределения данных 1\1 было выбрано 1\5


Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [7]:
from datasets import concatenate_datasets
imdb = load_dataset("imdb") ## загрузка дополнительных данных, которые в итоге не были использованы
tokenized_imdb = imdb.map(preprocess, batched=True)
dataset_cc = concatenate_datasets([tokenized_imdb['train'], tokenized_imdb['train']])
tokenized_imdb_split = dataset_cc.train_test_split(test_size=0.2)
tokenized_imdb_split

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 40000
    })
    test: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 10000
    })
})

In [9]:
training_args = TrainingArguments(
    output_dir="multi_class_classification_bert",
    learning_rate=2e-5,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=4,
    num_train_epochs=2, ## датасет достаточно обширный, 2 эпох для этой модели оказалось достаточно
    eval_steps = 4000,
    save_steps = 4000,
    weight_decay=0.01,
    evaluation_strategy="steps",
    save_strategy="steps",
    load_best_model_at_end=True,
    logging_steps = 50,
    report_to = "wandb", ## набор инструментов в основном для построения графиков прямо в процессе обучения (Weights and biases)
    save_total_limit=1,
    fp16 = False,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_text_split["train"],
    eval_dataset=tokenized_text_split["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics = None ## если оставить None, то будет использоваться кросс-энтропия
)

In [10]:
trainer.train()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: bossprocool (memers). Use `wandb login --relogin` to force relogin


  0%|          | 0/40000 [00:00<?, ?it/s]

You're using a DebertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


KeyboardInterrupt: 

In [10]:
trainer.evaluate()

  0%|          | 0/1250 [00:00<?, ?it/s]

{'eval_loss': 1.2642443180084229, 'eval_runtime': 108.6392, 'eval_samples_per_second': 46.024, 'eval_steps_per_second': 11.506, 'epoch': 0.63}


{'eval_loss': 1.2642443180084229,
 'eval_runtime': 108.6392,
 'eval_samples_per_second': 46.024,
 'eval_steps_per_second': 11.506,
 'epoch': 0.63}

In [9]:
trainer.save_model("tuned_multiclass_classification_actual") ## удобно, что можно с помощью того же самого обьекта сохранить модель

Интересные мысль: изначальная идея решения была в том, чтобы задачу присвоения оценки (1-10) представить в качестве задачи регрессии. Но для последующих предсказаний натренированную модель оказалось достаточно непросто использовать (в то время как для этой это решается в пару строк кода), поэтому было принято решение так не делать. Также была тренирована модель для бинарной классификации (положительный\отрицательный отзыв), но на сайте используется другой подход: оценка 1-4 - негативный отзыв, 7-10 - положительный (см. сайт). На дополнительной модели была достигнута функция потерь = 0.1